# Importaciones

In [13]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [34]:
#Configuraciones
from data_utils.configuraciones.variables import variables_data

#Manipulacion de datos
from data_utils.manipulacion_de_datos.maniuplacion_de_datos import  procesar_descripcion,   \
                                                                    estandarizado_columnas, \
                                                                    cargar_datos,           \
                                                                    generar_descripcion,    \
                                                                    procesar_descripcion,   \
                                                                    limpiar_datos
from data_utils.manipulacion_de_datos.gestion_de_datos import indexar_productos

#Funciones de Busquedas
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal
from data_utils.funciones_de_busqueda.busqueda_por_columna  import busqueda_codigo_por_columna

#Aplicaciones de Inteligencia Artificial
from data_utils.utilidades_comunes.procesamiento_de_lenguaje_natural.index import procesar_comparacion



variables_data = variables_data()

# Proceso LOAD DATASETS


Procesamiento df_MasterProducto_Filtro

In [15]:
df_Producto = cargar_datos(variables_data.csv_producto)
df_Producto = limpiar_datos(df_Producto)
df_Producto = indexar_productos(df_Producto,variables_data.index_productos)
df_MasterProductos = generar_descripcion(df_Producto,variables_data.name_columna_1_MasterProductos,variables_data.name_columna_2_MasterProductos,variables_data.descripcion_limpia_MasterProductos) 
df_MasterProductos = procesar_descripcion(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)

# El usuario ingresa el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ").strip()
# Filtrar productos por laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\manipulacion_de_datos\maniuplacion_de_datos.py:15: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(ruta_csv)


Se encontraron 38 productos coincidentes del laboratorio 'ABBVIE'.


In [16]:
df_MasterProductos_Filtro.columns

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto'],
      dtype='object')

Procesamiento df_Proveedor

In [17]:
#Carga Proveedor
cvs_proveedor = 'ABBVIE.csv'
df_Proveedor = cargar_datos(cvs_proveedor)
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = limpiar_datos(df_Proveedor)
#Indexacion 
df_Proveedor = indexar_productos(df_Proveedor,variables_data.index_proveedor)
#Normalizacion de Columnas
df_Proveedor = estandarizado_columnas(df_Proveedor,variables_data.name_columna_1_Proveedor,variables_data.name_columna_2_Proveedor,variables_data.descripcion_limpia_Proveedor)
#Normalizacion de Columna Descripcion
df_Proveedor = procesar_descripcion(df_Proveedor, variables_data.descripcion_limpia_Proveedor)

In [18]:
df_Proveedor.columns

Index(['condición de precios', 'cod_producto_proveedor', 'descripcion',
       'presentacion', 'unidades ', 'precio s/iva', 'descuento', 'total',
       'cod_barra', 'index_proveedor', 'descripcion_limpia_proveedor'],
      dtype='object')

# Proceso Busqueda por CODIGO DE PRODUCTO

In [35]:
df_master = pd.read_csv('df_MasterProductos_Filtro.csv')
df_proveedor = pd.read_csv('ABBVIE.csv')


df_master[variables_data.columna_codigo_producto_master] = df_master[variables_data.columna_codigo_producto_master].astype(str)
df_proveedor[variables_data.columna_codigo_producto_proveedor] = df_proveedor[variables_data.columna_codigo_producto_proveedor].astype(str)

In [36]:
df_proveedor_codigo_PRODUCTO = busqueda_codigo_por_columna(df_master, df_proveedor, 
                                                           variables_data.columna_codigo_producto_master, 
                                                           variables_data.columna_codigo_producto_proveedor)
print(f'DF POR CODIGO DE PRODUCTO')
df_proveedor_codigo_PRODUCTO

Se inicia proceso de búsqueda por código de barras en la columna cod_producto_proveedor
Se encontraron coincidencias
DF POR CODIGO DE PRODUCTO


,CONDICIÓN DE PRECIOS,cod_producto_proveedor,descripcion,Unnamed: 3,Unnamed: 4,presentacion,Unidades,Unnamed: 7,Precio S/IVA,Unnamed: 9,Descuento,Unnamed: 11,Total,cod_barra,cod_prod,niprod
0,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
1,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
2,NaN,20021980,HUMIRA AC x 1 (80 mg),NaN,NaN,"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
3,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,NaN,NaN,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,NaN,"2.345.085,38",NaN,18%,NaN,"1.987.360,49",NaN,NaN,NaN
4,NaN,20069005,RINVOQ 30mg,NaN,NaN,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
5,NaN,20078725,RINVOQ 45mg,NaN,NaN,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
6,NaN,20069017,SKYRIZI 150mg,NaN,NaN,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,NaN,NaN,NaN,-,8.054083e+12,NaN,NaN
7,NaN,20079636,SKYRIZI 360mg,NaN,NaN,2.4ML OBI AR FP,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
8,NaN,20079635,SKYRIZI 600 mg,NaN,NaN,10.0 mL VIAL AR FP,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN
9,NaN,42883,MAVIRET,NaN,NaN,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,42883,119555.0


# Proceso Busqueda por Codigo de Barra

In [21]:
df_MasterProductos_Filtro_cod_barra = pd.read_csv('df_MasterProductos_Filtro.csv')
df_Proveedor_cod_barra = pd.read_csv('ABBVIE.csv')
df_MasterProductos_Filtro_cod_barra['index_producto'] = df_MasterProductos_Filtro_cod_barra.index
df_Proveedor_cod_barra['index_proveedor'] = df_Proveedor_cod_barra.index

In [22]:
columna_codigo_de_barras_proveedor = 'cod_barra'
columna_codigo_de_barras_producto = 'codbarra'

df_proveedor_codigo_BARRA = busqueda_codigo_por_columna(df_master, df_proveedor, 
                                                        columna_codigo_de_barras_producto, 
                                                        columna_codigo_de_barras_proveedor)
print(f'DF POR CODIGO DE BARRAS')
df_proveedor_codigo_BARRA

Se inicia proceso de búsqueda por código de barras en la columna cod_barra
Se encontraron coincidencias
DF POR CODIGO DE BARRAS


,CONDICIÓN DE PRECIOS,cod_producto_proveedor,descripcion,Unnamed: 3,Unnamed: 4,presentacion,Unidades,Unnamed: 7,Precio S/IVA,Unnamed: 9,Descuento,Unnamed: 11,Total,cod_barra,codbarra,niprod
0,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
1,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
2,NaN,20021980,HUMIRA AC x 1 (80 mg),NaN,NaN,"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
3,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,NaN,NaN,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,NaN,"2.345.085,38",NaN,18%,NaN,"1.987.360,49",NaN,NaN,119569.0
4,NaN,20069005,RINVOQ 30mg,NaN,NaN,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
5,NaN,20078725,RINVOQ 45mg,NaN,NaN,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
6,NaN,20069017,SKYRIZI 150mg,NaN,NaN,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,NaN,NaN,NaN,-,8.054083e+12,8.054083e+12,119555.0
7,NaN,20079636,SKYRIZI 360mg,NaN,NaN,2.4ML OBI AR FP,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
8,NaN,20079635,SKYRIZI 600 mg,NaN,NaN,10.0 mL VIAL AR FP,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0
9,NaN,42883,MAVIRET,NaN,NaN,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,119569.0


# Proceso Busqueda NLP DESCRIPCION

In [23]:
df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
df_resultado_similiridad

,similaridad,condición de precios_x,cod_producto_proveedor_x,descripcion_x,presentacion_x,unidades _x,precio s/iva_x,descuento_x,total_x,cod_barra_x,...,condición de precios_y,cod_producto_proveedor_y,descripcion_y,presentacion_y,unidades _y,precio s/iva_y,descuento_y,total_y,cod_barra_y,index_proveedor_y
0,0.780209,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,NaN,...,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,NaN,0
1,0.606657,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,NaN,...,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,NaN,1
2,0.944039,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,NaN,...,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,NaN,2
3,0.760044,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",NaN,...,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",NaN,3
4,0.800373,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,NaN,...,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,NaN,4
5,0.639903,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,NaN,...,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,NaN,5
6,0.876686,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,8.054083e+12,...,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,8.054083e+12,6
7,0.352721,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,NaN,...,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,NaN,7
8,0.669698,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,NaN,...,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,NaN,8
9,0.595370,NaN,42883,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,NaN,...,NaN,42883,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,NaN,9


Procesamiento para comparacion de  Descripciones

In [24]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] > 0.8][['similaridad', 'niprod', 'cod_prod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'index_proveedor_y','codbarra']]

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto,index_proveedor_y,codbarra
2,0.944039,119550.0,55604,humira ac 1 80 miligramos 1 lapicera prellenad...,humira ac 1 80 miligramos 80 miligramos 08 mil...,2,8054083017648
4,0.800373,124959.0,59751,rinvoq 30 miligramos 30 miligramos 30 comprimi...,rinvoq 30 miligramos comp lib prolongada 30,4,8054083021386.0
6,0.876686,124898.0,59720,skyrizi 150 miligramos 1 lapicera prellenada p...,skyrizi 150 miligramos lapicera prellenada 1,6,8054083021935.0
10,0.852467,119575.0,53119,venclexta kit inicio kit de inicio 1 pack 4 7 ...,venclexta kit de inicio 1,10,8054083013343.0
11,0.943522,119573.0,53120,venclexta 100 miligramos 100 miligramos 120 co...,venclexta 100 miligramos comprimidos 120,11,8054083013336.0
14,0.812576,119561.0,14916,sevorane 250 mililitros,sevorane envase 250 mililitros,14,8054083015156.0
17,1.000000,119577.0,44168,zemplar 2 microgramo 30 capsulas,zemplar 2 microgramo capsulas 30,17,8054083012032.0


INDEX DE SUSTRACCION PARA EL DF PROVEEDOR

In [25]:
#Sutraccion de productos sobre df ORIGINAL DE PROVEEDORES.
similiridad = 'similaridad'

umbral_similaridad = 0.9  # Ajusta este valor según tu caso
df_filtrado_similaridad = df_resultado_similiridad[df_resultado_similiridad[similiridad] > umbral_similaridad]

# Ahora, con los índices originales, extraemos los productos del df_Proveedor
# Filtramos por 'indice_original_proveedor' para obtener solo las filas que superaron el umbral
df_Proveedor_Similiridad_filtrado = df_Proveedor[df_Proveedor[variables_data.index_proveedor].isin(df_filtrado_similaridad['index_proveedor_x'])]

# Mostrar los productos filtrados del proveedor
df_Proveedor_Similiridad_filtrado


,condición de precios,cod_producto_proveedor,descripcion,presentacion,unidades,precio s/iva,descuento,total,cod_barra,index_proveedor,descripcion_limpia_proveedor
2,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,NaN,2,humira ac 1 80 miligramos 1 lapicera prellenad...
11,NaN,20019067,VENCLEXTA 100mg,100 MG X 120 COMP.,NaN,NaN,NaN,NaN,8.054083e+12,11,venclexta 100 miligramos 100 miligramos 120 co...
17,NaN,20059449,ZEMPLAR 2 MCG,30 CAPSULAS,NaN,NaN,NaN,-,NaN,17,zemplar 2 microgramo 30 capsulas


In [26]:
df_resultado_similiridad.columns

#Arquitectura df_Productos

# *copia exacta de las columnas df_proveedor 
# xx_Niprod = Sistema Carena


Index(['similaridad', 'condición de precios_x', 'cod_producto_proveedor_x',
       'descripcion_x', 'presentacion_x', 'unidades _x', 'precio s/iva_x',
       'descuento_x', 'total_x', 'cod_barra_x', 'index_proveedor_x',
       'descripcion_limpia_proveedor', 'niprod', 'cod_prod', 'cod_abrev',
       'descrip1', 'descrip2', 'abrev', 'cod_catprod', 'codbarra',
       'cod_fbarra', 'unid_v', 'base_v', 'precio', 'mon_v', 'costo', 'base_c',
       'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c', 'unid_alt', 'rel_c_a',
       'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif', 'pped', 'st_min',
       'st_max', 'observ', 'pc_comision', 'pc_utilidad', 'ind_stock',
       'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto', 'ninum',
       'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item', 'cant_rep',
       'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto', 'condición de precios_y',
       'cod_producto_proveedor_y', 'descripcion_y', 'presentacion_

MODELO DE PROCESAMIENTOS DE LENGUAJE NATURAL

Bert

In [27]:

modelo_nombre_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

resultados_comparaciones = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BERT,                # modelo_nombre
    "BERT",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\procesamiento_de_lenguaje_natural\index.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master['numeros'] = df_master[columna_master].apply(extraer_numeros)  # Extrae números del master
c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\procesamiento_de_lenguaje_natural\index.py:115: SettingWithCopyWarning

In [28]:
resultados_comparaciones.columns


Index(['descripcion_proveedor', 'descripcion_match', 'similitud',
       'index_proveedor', 'index_producto', 'niprod'],
      dtype='object')

In [29]:
# Unir df_Proveedor con resultados_comparaciones en base a index_proveedor
df_resultado = df_Proveedor.merge(resultados_comparaciones, 
                                  left_index=True,  # Índice de df_Proveedor (index_proveedor debe coincidir con el índice)
                                  right_on='index_proveedor',  # Columna en resultados_comparaciones
                                  how='right')  # Solo los que están en resultados_comparaciones

# Reorganizar las columnas si es necesario
df_resultado = df_resultado[df_Proveedor.columns.tolist() + ['niprod', 'similitud']]

# Mostrar el resultado
df_resultado

,condición de precios,cod_producto_proveedor,descripcion,presentacion,unidades,precio s/iva,descuento,total,cod_barra,index_proveedor,descripcion_limpia_proveedor,numeros,niprod,similitud
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,NaN,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119551.0,0.901277
1,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,NaN,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119549.0,0.821588
2,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,NaN,0,humira ac 2 20 miligramos 2 jeringas prellenad...,"[2, 20, 2, 02]",119565.0,0.801296
3,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,NaN,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119552.0,0.862091
4,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,NaN,1,humira ac 2 40 miligramos 2 lapiceras prellena...,"[2, 40, 2, 04]",119554.0,0.811453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,NaN,19,boto vial 200 unidad,[200],119577.0,0.730224
59,NaN,93251AR,BOTOX,Vial 200 U,NaN,NaN,NaN,-,NaN,19,boto vial 200 unidad,[200],119567.0,0.721654
60,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,NaN,20,ozurde 07 miligramos c 1,"[07, 1]",130998.0,0.851499
61,NaN,93348AR,OZURDEX,0.7 MG C/1,NaN,NaN,NaN,-,NaN,20,ozurde 07 miligramos c 1,"[07, 1]",119568.0,0.838913


In [30]:
# Unir df_resultado con masterProductos en base a la columna niprod
df_resultado = df_resultado.merge(df_MasterProductos_Filtro[['niprod', variables_data.descripcion_limpia_MasterProductos]], 
                                  on='niprod',  # Clave de unión
                                  how='left').copy()  # 'left' para no perder filas en df_resultado

# Mostrar el resultado con la nueva columna agregada
#df_resultado[['niprod',"cod. producto",'presentacion','index_producto','similitud','index_proveedor']]


In [31]:
df_MasterProductos['xx_codigo_producto'] = None 
df_MasterProductos.columns

#Modelo del Nuevo csv del proveedor  para agregar a la master producto, si no se supera el umbral.
 
# niprod            = niprod  --> esto lo crea el sistema carena  por lo que no lo teng que crear
# descripcion       = descrip1
# presentacion      = descrip2	
# laboratorio       = atrib0
# codigo_producto   = codigo_producto  -> columna que se debe crear en la  MasterProducto



Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto', 'xx_codigo_producto'],
      dtype='object')

In [32]:
df_resultado.columns



Index(['condición de precios', 'cod_producto_proveedor', 'descripcion',
       'presentacion', 'unidades ', 'precio s/iva', 'descuento', 'total',
       'cod_barra', 'index_proveedor', 'descripcion_limpia_proveedor',
       'numeros', 'niprod', 'similitud', 'descripcion_limpia_producto'],
      dtype='object')

In [33]:
df_temp = df_resultado[['niprod', 'descripcion', 'presentacion', 'cod. producto']].copy()

# Renombrar columnas
df_temp.rename(columns={
    'descripcion': 'descrip1',
    'presentacion': 'descrip2',
    'cod. producto': 'xx_codigo_producto'
}, inplace=True)

df_temp.columns

KeyError: "['cod. producto'] not in index"

In [28]:
# Unir ambos DataFrames sin filtrar por similitud
df_MasterProductos = pd.concat([df_MasterProductos, df_temp], ignore_index=True)
df_MasterProductos[['descrip1', 'descrip2']].iloc[57176:57185]

,descrip1,descrip2


PubMedBERT

In [29]:
"""
modelo_nombre_PubMedBert = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

resultados_PubMedBERT = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_PubMedBert,                # modelo_nombre
    "PubMedBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
"""

'\nmodelo_nombre_PubMedBert = \'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract\'\n\nresultados_PubMedBERT = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_PubMedBert,                # modelo_nombre\n    "PubMedBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n'

BioBert

In [30]:
"""
modelo_nombre_BioBert = 'dmis-lab/biobert-base-cased-v1.1'

resultados_BioBert = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BioBert,                # modelo_nombre
    "BioBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
 """

'\nmodelo_nombre_BioBert = \'dmis-lab/biobert-base-cased-v1.1\'\n\nresultados_BioBert = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_BioBert,                # modelo_nombre\n    "BioBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n '

FINE TUNING